# Parse XML annotation file with X,Y coordinates and instance ID into a DataFrame

In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import os

In [2]:
def xml_to_df(xml_filepath):
    append_df = []
    tree = ET.parse(xml_filepath)
    root = tree.getroot()
    for Annotation in root.iter("Annotation"):
        for Region in Annotation.iter('Region'): #iterate over the Region so we can iterate over id 1 and 2 (two circles):
            x = np.array([Vertex.get('X') for Vertex in Region.iter('Vertex')])
            y = np.array([Vertex.get('Y') for Vertex in Region.iter('Vertex')])
            id = np.array([Region.get('Id')])
            coord_dict = {"X": [x], "Y": [y], "ID": [id]}
            df = pd.DataFrame(data = coord_dict)
            df.ID = df.ID.astype(int)
            append_df.append(df)
    coord_df = pd.concat(append_df)
    return(coord_df)

In [3]:
# # for pc:
# # src = r'\\fatherserverdw\kyuex\clue images'
# # for mac:
# src = r'//Volumes/kyuex/clue images'
# xml = [_ for _ in os.listdir(src) if _.endswith('xml')]
# xml

coord_df = xml_to_df('//Volumes/kyuex/clue images/2022-06-07 13.18.40.xml')
coord_df

,X,Y,ID
0,"[5615, 5725, 5810, 5865, 5900, 5915, 5930, 594...","[10850, 10890, 10930, 10950, 10960, 10970, 109...",1
0,"[24299, 24364, 24483, 24537, 24613, 24624, 246...","[20025, 20068, 20133, 20176, 20209, 20231, 202...",2


# Convert X,Y Coordinates to Binary Mask

In [4]:
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2

def df_to_mask(coord_df):
    for i in np.arange(len(coord_df)):
        xx = coord_df.iloc[i].X
        yy = coord_df.iloc[i].Y
        xy = list(zip(xx,yy))
        contours = np.array(xy)
        image = np.full((40000, 40000), 255, dtype = np.uint8) #white
        mask = cv2.drawContours(image,[contours.astype(int)],-1,(0,255,0),10) #contourIdx = -1, draw all contours
        #cv2.imwrite('//Volumes/Kevin/' + 'mask' + str(i) + '.jpg',mask) # save image on drive
        return mask
df_to_mask(coord_df)

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

# Cropping RGB Image with Mask:

In [5]:
import openslide
# Get basic information of the slide we have:
slide = openslide.open_slide('//Volumes/kyuex/clue images/2022-06-07 13.18.40.ndpi')
rgb_dim = slide.dimensions
print("Dimension of level 0 of the image is: ",rgb_dim)
slide_level_dim = slide.level_dimensions
num_levels = len(slide_level_dim)
print("Number of levels in this image are:",num_levels)
factors = slide.level_downsamples
print("Each level is downsampled by:",factors)

Dimension of level 0 of the image is:  (48000, 47872)
Number of levels in this image are: 8
Each level is downsampled by: (1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0)


In [6]:
from openslide.deepzoom import DeepZoomGenerator
# get tiles now:
tiles = DeepZoomGenerator(slide, tile_size = 256, overlap = 0, limit_bounds = False)
# each tile has size 256 x 256 image
print("Number of levels in this tiles object are:",tiles.level_count)
print("Dimensions of data in each level are:",tiles.level_dimensions)
print("Total number of tiles are:",tiles.tile_count)

Number of levels in this tiles object are: 17
Dimensions of data in each level are: ((1, 1), (2, 2), (3, 3), (6, 6), (12, 12), (24, 24), (47, 47), (94, 94), (188, 187), (375, 374), (750, 748), (1500, 1496), (3000, 2992), (6000, 5984), (12000, 11968), (24000, 23936), (48000, 47872))
Total number of tiles are: 46979


In [8]:
print("Total number of tiles are:", tiles.tile_count)
max_tile_count_in_selected_image = tiles.level_tiles[9]
print("Total number of tiles in the large image are:", max_tile_count_in_selected_image)
single_tile = tiles.get_tile(9,(0,0))
single_tile_RGB = single_tile.convert('RGB')
single_tile_RGB.show()

Total number of tiles are: 46979
Total number of tiles in the large image are: (2, 2)


In [9]:
mask = df_to_mask(coord_df)
print("Dimensions of mask is:",mask.shape)
single_tile_dim = tiles.get_tile_dimensions(9,(0,0))
print("Dimensions of tile is:",single_tile_dim)
mask_x, mask_y = mask.shape
tile_x, tile_y = single_tile_dim

Dimensions of mask is: (40000, 40000)
Dimensions of tile is: (256, 256)


In [10]:
#np.unique(mask, return_counts = True)
line = np.where(mask == 0)
x1 = max(line[0])
y1 = max(line[1])
x1,y1

(13975, 14235)

In [12]:
mask_cropped = mask[0:14236,0:14236]
mask_cropped = cv2.resize(mask_cropped,(256,256))
cropped_image = cv2.bitwise_and(single_tile_RGB, mask_cropped)
# error because single_tile_RGB array is three tuples while mask is just one number

error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'bitwise_and'
> Overload resolution failed:
>  - src1 is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src1'
